In [1]:
import openai 
from openai import OpenAI
import os
from similarDocSearch import *
import json
from credentials import *

# Getting all the required credentials

In [2]:
cred = Credentials()
all_cred = cred.get_credential()

# Reading the data file

In [3]:
with open("data/documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

len(docs_raw) # # as it's of 3 courses

3

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course'] 
        documents.append(doc)

In [5]:
documents[:2]

[{'text': "The purpose of this document is to capture frequently asked\ntechnical question\nThe next cohort starts in Jan 2025. More inFo at DTC Article.\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - See DE-zoomcamp  prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

# Retrieving the top n documents matching the user's passed query

In [6]:
# Initializing the class similarDocSearch
obj = similarDocSearch(text_fields = ["question", "text", "section"])
obj

In [7]:
# Fitting
obj.fit(documents)

In [8]:
# Giving a query to get the top n documents matching to our query
q = "The course has already started, can i still enroll?"
boost = {'question': 3, 
         'section': 0.5} # Used to give a field more or less importance 
filter_course = "data-engineering-zoomcamp"
num_results = 5

obj.search(query = q, boosts=boost, filter_course=filter_course, num_results=num_results)

[{'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - See DE-zoomcamp  prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'All the main videos are stored in the Main “

In [9]:
client = OpenAI(api_key=all_cred["RAG_OPENAI_KEY"])

In [11]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role":"user", "content":"how many days are there in a week"}]
)

response

ChatCompletion(id='chatcmpl-A3RON4M3AJvfrueuwtZuBUYrja9AT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='There are 7 days in a week.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725383643, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f33667828e', usage=CompletionUsage(completion_tokens=9, prompt_tokens=15, total_tokens=24))

In [16]:
response.choices[0].message.content

'There are 7 days in a week.'